In [1]:
import os
import sys
import pandas as pd
import numpy as np
import json
import copy
import itertools
import tensorflow as tf
from KT.preprocess.data_utils import read_data,get_max_concepts,calStatistics,extend_multi_concepts,id_mapping,save_id2idx,train_test_split,save_dcur,generate_sequences, get_mask_tokens
import pickle
import glob
import matplotlib.pyplot as plt
import time
from KT.models.model_for_kt import TFTransfoXLModel,TFTransfoXLLMHeadModel,TFTransfoXLMLMHeadModel

from transformers import TransfoXLConfig
from tensorflow.keras.utils import register_keras_serializable
import datetime
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


2024-01-15 18:59:04.371087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 18:59:04.888615: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-01-15 18:59:04.888662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-01-15 18:59:04.888667: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:

config_xl = TransfoXLConfig(
    data = '/home/jun/workspace/KT/data/ednet/data.txt',
    dataset = 'wt103',
    d_embed=128,
    d_head = 32,
    d_model=128,
    mem_len=400,
    n_head=8,
    n_layer=6,
    batch_size = 1,
    tgt_len = 140,
    ext_len = 0,
    eval_tgt_len = 36,
    eos_token=2,
    num_c=123,
    mask_token=3,
    C_vocab_size=188,
    R_vocab_size = 2
)

In [3]:
model = TFTransfoXLMLMHeadModel(config=config_xl)
model.load_weights('/home/jun/workspace/KT/save_model/6ep_600mem.ckpt/my_checkpoint')


2024-01-15 18:59:06.954103: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-15 18:59:06.959108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-15 18:59:06.959469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-15 18:59:06.960204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [4]:

csv_file_path = '/home/jun/workspace/KT/data/ednet/test_mapping.csv' # 이 값을 사용자가 넣을 수 있도록 하자

df= pd.read_csv(csv_file_path)

In [5]:
question_list = df.loc[df['uid'] == 3600, 'questions']
concepts_list = df.loc[df['uid'] == 3600, 'concepts']
responses_list = df.loc[df['uid'] == 3600, 'responses']

In [6]:
question_list = [int(x) for x in question_list[0].split(',')]
concepts_list = [int(x) for x in concepts_list[0].split(',')]
responses_list = [int(x) for x in responses_list[0].split(',')]


In [7]:
masked_R, labels = get_mask_tokens(responses_list,config_xl.mask_token)


In [8]:
masked_R

<tf.Tensor: shape=(280,), dtype=int32, numpy=
array([1, 0, 1, 1, 0, 0, 0, 1, 0, 3, 1, 1, 1, 1, 0, 0, 1, 0, 3, 3, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3,
       1, 1, 1, 3, 0, 0, 3, 0, 0, 0, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 3,
       3, 3, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 3, 1, 3, 3, 0, 3, 0, 1, 3, 0, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 3, 0, 0, 0, 1, 1, 3, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 0, 0, 1, 3, 1, 0, 1, 3, 3, 0, 1,
       1, 3, 1, 1, 3, 3, 1, 1, 3, 3, 1, 1, 0, 1, 1, 3, 1, 1, 1, 1, 3, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 3, 3, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 3, 1, 1], dtype=int32)>

In [9]:



# 여기서 읽어온 데이터를 딕셔너리키 가지고 변환해서 ip mapping 하고 

n_step = len(concepts_list) // (config_xl.tgt_len)

sliced_qseqs = tf.slice(question_list,[0],[n_step *config_xl.tgt_len])  
sliced_cseqs = tf.slice(concepts_list,[0],[n_step *config_xl.tgt_len])  
sliced_masked_R = tf.slice(masked_R,[0],[n_step *config_xl.tgt_len]) 
sliced_labels = tf.slice(labels,[0],[n_step *config_xl.tgt_len]) 


new_shape = (config_xl.batch_size, -1)  # 나머지 차원은 자동으로 계산됨

qseq_reshaped = tf.reshape(sliced_qseqs, new_shape)
cseq_reshaped = tf.reshape(sliced_cseqs, new_shape)
masked_R_reshaped = tf.reshape(sliced_masked_R, new_shape)
labels_reshaped = tf.reshape(sliced_labels, new_shape)


qseq_transposed = tf.transpose(qseq_reshaped)
cseq_transposed = tf.transpose(cseq_reshaped)
masked_R_transposed = tf.transpose(masked_R_reshaped)
labels_transposed = tf.transpose(labels_reshaped)

In [10]:
predict_dataset = tf.data.Dataset.from_tensor_slices(
    (cseq_transposed, masked_R_transposed,labels_transposed))

In [11]:
predict_dataset =predict_dataset.batch(config_xl.tgt_len)

In [12]:
sliced_qseqs

<tf.Tensor: shape=(280,), dtype=int32, numpy=
array([4860, 6211, 9250, 4461, 3240, 3133, 6966, 9852, 3535,  672, 1302,
       3151, 6326, 1818, 1045, 1018, 1035, 4633, 3472,  670, 1041, 2115,
       1375, 6743, 1170, 1169, 1168, 1171, 4669, 2153, 2155, 2152, 2154,
       7403, 7403, 7403, 7403, 3380,   51,   51,   51,   50,   50,   50,
         52,   52,   52, 2212, 2212, 2212, 2210, 2210, 2210, 2211, 2211,
       2211, 1830, 1830, 1830, 1831, 1831, 1831, 1831, 1829, 1829, 1829,
       1887, 1885, 1888, 1886, 3206, 3206, 3206, 3207, 3207, 3207, 3205,
       3205, 3205,  264,  264,  264,  264, 5594, 5594, 5594, 5593, 5593,
       5593, 5594, 5594, 5594, 5593, 5593, 5593, 5595, 5595, 5595, 3737,
       3737, 3737, 3737, 3381, 3381, 3381, 3813, 3813, 3813, 3813,  949,
         10,   19, 4853, 2498, 2132, 2134, 2133, 2135,  623,  621,  624,
        622, 1169, 1170, 1168, 1171, 1867, 1866, 1865, 1868, 1906, 1906,
       1907, 1907, 1905, 1905, 1908, 1908, 1856, 1856, 1853, 1853, 1854,
     

In [13]:
mems =None
predictions = []
for test_data1, test_data2, test_target in predict_dataset:
        outputs = model(concepts=test_data1, responses=test_data2, labels=test_target, mems=mems)
        loss = outputs.loss
        mems = outputs.mems
        
        reshape = tf.reshape(loss, [-1, config_xl.R_vocab_size])
        predicted_labels = tf.argmax(reshape, axis=1)
        predictions.append(predicted_labels.numpy().tolist())


In [14]:
flattened_list = [item for sublist in predictions for item in sublist]
flattened_list

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [15]:
sliced_labels

<tf.Tensor: shape=(280,), dtype=int32, numpy=
array([-100, -100, -100, -100, -100, -100, -100, -100, -100,    1, -100,
       -100, -100, -100, -100, -100, -100, -100,    0,    1, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100,    0, -100, -100, -100, -100, -100, -100, -100,
       -100, -100,    1, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100,    1,    1,
       -100, -100, -100,    1, -100, -100,    0, -100, -100, -100, -100,
       -100, -100, -100,    1, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    1,
          1,    0, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
          1,    1, -100,    1,    0, -100,    0, -100, -100,    1, -100,
     

<tf.Tensor: shape=(280,), dtype=int32, numpy=
array([-100, -100, -100, -100, -100, -100, -100, -100, -100,    1, -100,
       -100, -100, -100, -100, -100, -100, -100,    0,    1, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100,    0, -100, -100, -100, -100, -100, -100, -100,
       -100, -100,    1, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100,    1,    1,
       -100, -100, -100,    1, -100, -100,    0, -100, -100, -100, -100,
       -100, -100, -100,    1, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    1,
          1,    0, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
          1,    1, -100,    1,    0, -100,    0, -100, -100,    1, -100,
     

In [21]:
q

<tf.Tensor: shape=(44,), dtype=int32, numpy=
array([ 672, 3472,  670, 7403,   52, 1829, 1829, 1886, 3206,  264,  949,
         10,   19, 1907, 1907, 1905, 1908, 1856, 1853, 1854, 1197, 1164,
       2137, 2399, 2400, 1883, 1176, 1888, 1886, 6152, 1978, 1978, 1977,
       1977,  643,  642, 1977,   26,  627, 1881, 1883, 1883, 1181, 1901],
      dtype=int32)>

In [22]:
c

<tf.Tensor: shape=(44,), dtype=int32, numpy=
array([113, 101, 120,  49,  49,  43,   8,  26,   9, 132, 134,  12,  24,
        30, 138,  29,  30,  30, 122,   4,  30,  30,  30,  25,  29,  30,
        29,  29,  26,  29,  30,  29,  30,  25,  22,  30,  25,  30,  25,
        30,  30,  10,  30, 114], dtype=int32)>

In [24]:
df = pd.DataFrame({'Question':sliced_qseqs,'Concepts':sliced_cseqs,'Responses':flattened_list})

,Question,Concepts,Responses
0,4860,121,1
1,6211,27,1
2,9250,31,1
3,4461,110,1
4,3240,114,1
5,3133,26,1
6,6966,10,1
7,9852,57,1
8,3535,35,1
9,672,113,1


In [7]:
A = 5
B=2
print('this is {} {}'.format(A,B))

this is 5 2
